## Control the Lego Boost using a Game Controller

Run the cells below to import the needed libraries, as well as initialize the controller. Buttons are corresponding to a 8BitDo controller.

In [1]:
# uncomment the line below to download ipywidgets
# !pip install ipywidgets

In [2]:
import ipywidgets as wid
from pylgbst.hub import MoveHub, VisionSensor, TiltSensor
from pylgbst import get_connection_bleak
import time
import random

Make sure to add the Bluetooth address of your MoveHub, which can be found in settings. Before running the cell below, press the button on your MoveHub such that the LED is flashing, this means its looking for a connection. Once the connection is stable, the LED will turn blue.

In [3]:
# connecting to the MoveHub using its individual Bluetooth address 
conn = get_connection_bleak(hub_mac='00:16:53:C3:C2:4F', hub_name=MoveHub.DEFAULT_NAME)
hub = MoveHub(conn)

Have no dedicated class for peripheral type 0x42 (UNKNOWN) on port 0x46


In [4]:
# initializing the controller
gamepad = wid.Controller()

In [5]:
def on_sent(data):
    if gamepad.connected:
        # moving forward (right joystick)
        if data['owner'].axes[3].value < 0:
            hub.motor_AB.timed(1, 0.1, 0.1)
            
        # moving backwards (right joystick)
        if data['owner'].axes[3].value > 0:
            hub.motor_AB.timed(1, -0.1, -0.1)
            
        # moving to the right (right joystick)
        if data['owner'].axes[0].value < 0:
            hub.motor_B.timed(1, 0.1)
            
        # moving to the left (right joystick)
        if data['owner'].axes[0].value > 0:
            hub.motor_A.timed(1, 0.1)
            
        # speeding forward (R2 button)
        if data['owner'].buttons[7].value:
            hub.motor_AB.timed(1, 1, 1)
        
        # stoping the motors fully (L2 button)
        if data['owner'].buttons[6].value:
            hub.motor_external.stop()
        
        # changing led color (select button)
        if data['owner'].buttons[8].value:
            # randomly generated color
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            hub.led.set_color(color)
            
        # move in a square (B button)
        if data['owner'].buttons[0].value:
            for count in range(2):
                hub.motor_AB.timed(1, 0.1, 0.1)
                hub.motor_A.angled(540, 1)
        
        # move in a circle (A button)
        if data['owner'].buttons[1].value:
            for count in range(2):
                hub.motor_AB.timed(1, 0.1, 0.5)
        
        # rotate 360° (Y button)
        if data['owner'].buttons[2].value:
            hub.motor_A.timed(1.5, 1)
        
        # shoot arrow (X button)
        if data['owner'].buttons[3].value:
            hub.port_D.timed(0.3, 0.5)
            hub.port_D.timed(0.3, -0.4)
            
        # distance detection (up arrow) 
        if data['owner'].buttons[12].value:
            def callback(distance):
                print("Distance: %s" % (distance))
                
            hub.vision_sensor.subscribe(callback, mode=VisionSensor.DISTANCE_INCHES)
            time.sleep(2)# play with sensor while it waits
            hub.vision_sensor.unsubscribe(callback)
            
        # color detection (left arrow) 
        if data['owner'].buttons[14].value:
            def callback(color):
                print("Color: %s" % (color))
                
            hub.vision_sensor.subscribe(callback, mode=VisionSensor.COLOR_INDEX)
            time.sleep(2)# play with sensor while it waits
            hub.vision_sensor.unsubscribe(callback)
            
        # detect color and distance (down arrow)
        if data['owner'].buttons[13].value:
            def callback(color, distance):
                print("Color: %s / Distance: %s" % (color, distance))
                
            hub.vision_sensor.subscribe(callback, mode=VisionSensor.COLOR_DISTANCE_FLOAT)
            time.sleep(2)# play with sensor while it waits
            hub.vision_sensor.unsubscribe(callback)
            
        # detect 3-axis position (right arrow)
        if data['owner'].buttons[15].value:
            def callback(roll, pitch, yaw):
                print("Roll: %s / Pitch: %s / Yaw: %s" % (roll, pitch, yaw))
                
            hub.tilt_sensor.subscribe(callback, mode=TiltSensor.MODE_3AXIS_ACCEL)
            time.sleep(2)# play with sensor while it waits
            hub.tilt_sensor.unsubscribe(callback)

        # turn head to left (L1 button)
        if data['owner'].buttons[4].value:
            hub.port_D.timed(0.15, 0.4)
        
        # turn head to the right (R1 button)
        if data['owner'].buttons[5].value:
            hub.port_D.timed(0.15, -0.4)
            
gamepad.observe(on_sent)
gamepad

Controller(axes=(Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(value=0.0)), buttons=(Button(value=0.…

Run the cell below to disconnect from the MoveHub. 

In [6]:
hub.disconnect()

Hub disconnects
